## Softmax Classifier

In previous part, we have discussed about the binary classifier problems, which is done using *logistic regression*. However, we have to be aware that in real life case, the classification is always multi-class. In this case, we will use the **softmax classifier** to solve it.  

We will use MNIST 0-9 dataset as example in here ('hello world' in deep learning field).

Since the outputs of this model should be *probability* as well, which means that the output should follow the rules below:
$$
\forall n \Rightarrow P(\hat{y}=n) \geq 0
$$
Also, it is mandetoray that:
$$
\sum_{n=0}^9 P(\hat{y}=n)=1
$$
To achieve, we can use **softmax function**, which is shown as below (assume $Z^l \in \mathbb{R}^K$ is the output of the last linear layer):
$$
 P(y = i) = \frac{e^{z_i}}{\sum_{j=0}^{K-1} e^{z_j}}, \; i \in \{0, \ldots, K - 1\} 
$$

In [6]:
import torch
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F         # for using function relu()
import torch.optim as optim             # for using opt

### Step 1: Prepare Dataset

We have to convert the img from PIL img to Tensor:
$$
\mathbb{Z}^{28\times 28},pixel\in\{0,\cdots,255\}\Rightarrow \mathbb{R}^{1\times28\times28},pixel\in [0,1]
$$
Therefore we need to convert the initial pixel value to *Tensor* and *Normalized Value*.

In [7]:
batch_size = 64
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]) # 归一化,均值和方差
 
train_dataset = datasets.MNIST(root='../dataset/mnist/', 
                               train=True, 
                               download=False, 
                               transform=transform)
train_loader = DataLoader(train_dataset, 
                          shuffle=True, 
                          batch_size=batch_size)

test_dataset = datasets.MNIST(root='../dataset/mnist/', 
                              train=False, 
                              download=False, 
                              transform=transform)
test_loader = DataLoader(test_dataset, 
                         shuffle=False, 
                         batch_size=batch_size)

### Step 2: Build Network

In [8]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.l1 = torch.nn.Linear(784, 512)
        self.l2 = torch.nn.Linear(512, 256)
        self.l3 = torch.nn.Linear(256, 128)
        self.l4 = torch.nn.Linear(128, 64)
        self.l5 = torch.nn.Linear(64, 10)
 
    def forward(self, x):
        x = x.view(-1, 784)  # -1其实就是自动获取mini_batch
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return self.l5(x)  # 最后一层不做激活，不进行非线性变换
 
 
model = Net()

### Step 3: Build Loss Function and Opt

Keep in mind that the **softmax** has been included in the **CrossEntropyLoss()** already.

In [9]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

### Step 4: Training and Testing

In [10]:
def train(epoch):
    running_loss = 0.0
    for batch_idx, data in enumerate(train_loader, 0):
        # 获得一个批次的数据和标签
        inputs, target = data
        optimizer.zero_grad()
        # 获得模型预测结果(64, 10)
        outputs = model(inputs)
        # 交叉熵代价函数outputs(64,10),target（64）
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
 
        running_loss += loss.item()
        if batch_idx % 300 == 299:
            print('[%d, %5d] loss: %.3f' % (epoch+1, batch_idx+1, running_loss/300))
            running_loss = 0.0
 
 
def test():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, dim=1) # dim = 1 列是第0个维度，行是第1个维度
            total += labels.size(0)
            correct += (predicted == labels).sum().item() # 张量之间的比较运算
    print('accuracy on test set: %d %% ' % (100*correct/total))

if __name__ == '__main__':
    for epoch in range(10):
        train(epoch)
        test()

[1,   300] loss: 2.140
[1,   600] loss: 0.775
[1,   900] loss: 0.440
accuracy on test set: 89 % 
[2,   300] loss: 0.317
[2,   600] loss: 0.273
[2,   900] loss: 0.223
accuracy on test set: 94 % 
[3,   300] loss: 0.188
[3,   600] loss: 0.172
[3,   900] loss: 0.158
accuracy on test set: 95 % 
[4,   300] loss: 0.133
[4,   600] loss: 0.128
[4,   900] loss: 0.117
accuracy on test set: 96 % 
[5,   300] loss: 0.096
[5,   600] loss: 0.097
[5,   900] loss: 0.098
accuracy on test set: 97 % 
[6,   300] loss: 0.079
[6,   600] loss: 0.077
[6,   900] loss: 0.076
accuracy on test set: 96 % 
[7,   300] loss: 0.059
[7,   600] loss: 0.063
[7,   900] loss: 0.065
accuracy on test set: 97 % 
[8,   300] loss: 0.050
[8,   600] loss: 0.049
[8,   900] loss: 0.052
accuracy on test set: 97 % 
[9,   300] loss: 0.039
[9,   600] loss: 0.039
[9,   900] loss: 0.044
accuracy on test set: 97 % 
[10,   300] loss: 0.031
[10,   600] loss: 0.029
[10,   900] loss: 0.034
accuracy on test set: 97 % 
